# Imports/Installs

In [4]:
import warnings

warnings.filterwarnings('ignore')

In [ ]:
pip install selenium

In [6]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import json

## Scrapíng

In [ ]:
def tipo_estatisticas(elemento):

    teste = True
    
    #Seleciona tipo de estatística na tabela (defensivas, ofensivas ou passes). Segue o padrão 
    #'/html/body/div[4]/div[5]/ul/li[elemento]/a', em que o elemento varia de 2 a 4
    driver.find_element_by_xpath('/html/body/div[4]/div[5]/ul/li[{}]/a'.format(elemento)).click()

    time.sleep(2)

    #Seleciona todos os jogadores. Segue o padrão '/html/body/div[4]/div[5]/div[elemento]/div[1]/div[2]/dl[2]/dd[2]/a', 
    #em que o elemento varia de 3 a 5
    driver.find_element_by_xpath('/html/body/div[4]/div[5]/div[{}]/div[1]/div[2]/dl[2]/dd[2]/a'.format(elemento + 1)).click()

    time.sleep(2)
    
    return teste
    
def estrutura_df(paginas):
    
    df = pd.DataFrame()
    
    k = 1
    
    while k <= paginas:

        #Seleciona o conteúdo da tabela. Segue o padrão '/html/body/div[4]/div[5]/div[elemento]/div[3]', 
        #em que o elemento varia de 3 a 5
        element = driver.find_element_by_xpath('/html/body/div[4]/div[5]/div[{}]/div[3]'.format(elemento + 1))

        html_content = element.get_attribute('outerHTML')

        #Parsear o conteúdo HTML - BeautifulSoup
        soup = BeautifulSoup(html_content, 'html.parser')


        #Estruturando os dados em formato dataframe
        table = soup.find(name = 'table')

        df_full = pd.read_html(str(table))[0]

        #Separando a coluna 'Player.1' em suas informações apresentadas (jogador, time, idade e posição)

        df_full['posicao'] = df_full['Player.1'].str.split(', ').apply(lambda x: x[-1])
        df_full['idade'] = df_full['Player.1'].str.split(', ').apply(lambda x: x[-2])

        #Para definir o nome e o time de cada jogador de maneira adequada, uma maneira é comparar as listas após um split das duas
        #primeiras colunas do dataframe. Exemplo: '1Bruno HenriqueFlamengo' e 'Bruno Henrique Flamengo'. Comparando elemento a elemento,
        #é possível identificar em qual posição devemos cortar a lista para ter dados coerentes.

        lista_nomes = []
        lista_times = []

        for i in df_full.index:
            lista_coluna_1 = (str((i + 1)+10*(k-1)) + df_full['Player.1'][i].split(', ')[0]).split()
            lista_coluna_2 = df_full.Player[i].split(', ')[0].split()
            teste = False
            j = 0

            while teste == False:
                if lista_coluna_1[j] not in lista_coluna_2:
                    teste = True           
                else:
                    j += 1

            lista_nomes.append(' '.join(df_full['Player.1'][i].split(', ')[0].split()[0:j+1]))
            lista_times.append(' '.join(df_full['Player.1'][i].split(', ')[0].split()[j + 1:]))

        #Define as colunas de nomes e times como sendo as listas criadas
        df_full['nome'] = lista_nomes
        df_full['time'] = lista_times





        df = pd.concat([df, df_full], axis = 0)

        indice = range(df.shape[0])
        
        df.index = indice

        #Selecionaa próxima página da tabela. Segue o padrão '/html/body/div[4]/div[5]/div[elemento]/div[4]/div/dl[2]/dd[3]/a', 
        #em que o elemento varia de 3 a 5
        driver.find_element_by_xpath('/html/body/div[4]/div[5]/div[{}]/div[4]/div/dl[2]/dd[3]/a'.format(elemento + 1)).click()

        time.sleep(2)
        
        k += 1
    
    return df

#Define as ligas e o número de páginas de estatísticas dos jogadores de cada uma
ligas = {'brasil': {'url':'https://1xbet.whoscored.com/Regions/31/Tournaments/95/Seasons/8555/Stages/19551/PlayerStatistics/Brazil-Brasileir%C3%A3o-2021', 
                    'paginas':74}, 
         'argentina': {'url':'https://1xbet.whoscored.com/Regions/11/Tournaments/68/Seasons/8679/Stages/19893/PlayerStatistics/Argentina-Liga-Profesional-2021', 
                       'paginas': 78}}

#Define o controle do navegador em modo automático
option = Options()
option.headless = True
#Define o diretório do driver do Google Chrome (em cada máquina será diferente)
chrome_driver = 'C:/Users/m_hen/OneDrive/Área de Trabalho/Drivers/chromedriver' #necessário inverter o sentido da barra do path ('\' não estava reconhecendo)
driver = webdriver.Chrome(chrome_driver)

#Cria um dataframe para que será preenchido com os dados raspados já prontos
dataframe_final = pd.DataFrame()

for liga in ligas:
    
    #Cria um dicionário que gravará os dataframes iniciais (dados ainda não organizados), separados por tipo de estatística
    df_estatisticas = {}

    tipos = ['defensivas', 'ofensivas', 'passes']
    
    #Varre os elementos html necessários das páginas selecionadas
    for elemento in range(2,5):
        driver.get(ligas[liga]['url'])

        time.sleep(10)

        tipo_estatisticas(elemento)

        df_estatisticas[tipos[elemento - 2]] = estrutura_df(ligas[liga]['paginas'])

    #Define dicionário que será usado para renomear as colunas em português
    colunas = {'colunas_defensivas': {'Apps':'partidas', 'Mins':'minutos', 'Tackles':'desarmes', 'Inter':'interceptacoes', 
                                      'Fouls':'faltas_cometidas', 'Offsides':'impedimentos_ganhos', 'Clear':'afastadas_perigo', 
                                      'Drb':'dribles_sofridos', 'Blocks':'Blocks', 'OwnG':'gols_contra', 'Rating':'nota'}, 
               'colunas_ofensivas': {'Apps':'partidas', 'Mins':'minutos', 'Goals': 'gols', 'Assists': 'assistencias',
                                     'SpG':'chutes', 'KeyP': 'passes_chave', 'Drb':'dribles', 'Fouled':'faltas_sofridas', 
                                     'Off': 'impedimentos', 'Disp':'desarmes_sofridos','UnsTch':'dominios_errados','Rating':'nota'}, 
               'colunas_passes': {'Apps':'partidas', 'Mins':'minutos',  'Assists': 'assistencias','AvgP':'passes', 
                                  'KeyP': 'passes_chave', 'LongB':'bolas_longas', 'PS%':'perc_passes','ThrB':'enfiadas' ,
                                  'Crosses':'cruzamentos','Rating':'nota'}}

    for tipo in tipos:
        df_estatisticas[tipo].drop(columns = ['Player', 'Player.1'], inplace = True)
        df_estatisticas[tipo].rename(columns = colunas['colunas_{}'.format(tipo)], inplace = True)

    dataframe_liga = pd.merge(df_estatisticas['ofensivas'], df_estatisticas['defensivas'], on = ['nome', 'time', 'idade', 'posicao', 'partidas', 'minutos', 'nota'])
    dataframe_liga = pd.merge(dataframe_liga, df_estatisticas['passes'], on = ['nome', 'time', 'idade', 'posicao', 'partidas', 'minutos', 'nota', 'assistencias', 'passes_chave'])
    
    dataframe_final = pd.concat([dataframe_final, dataframe_liga], axis = 0)
    
driver.quit()

## Organização do dataframe e exportação para csv

In [ ]:
dataframe_final = dataframe_final[['nome', 'idade', 'time', 'posicao', 'partidas', 'minutos', 'gols', 'assistencias', 
                                   'chutes', 'passes_chave', 'dribles', 'faltas_sofridas', 'impedimentos', 
                                   'desarmes_sofridos', 'dominios_errados', 'desarmes', 'interceptacoes', 
                                   'faltas_cometidas', 'impedimentos_ganhos', 'afastadas_perigo', 'dribles_sofridos', 
                                   'Blocks', 'gols_contra', 'passes', 'perc_passes', 'cruzamentos', 'bolas_longas', 
                                   'enfiadas','nota']]

dataframe_final.to_csv('C:/Users/m_hen/OneDrive/Área de Trabalho/Projeto_ML/estatisticas_jogadores_2021.csv', ',', 
                       index = False)